# Helper Functions

In [ ]:
def get_pdf(data):
    from scipy.stats.kde import gaussian_kde
    from numpy import linspace
    
    kde = gaussian_kde(data)

    return kde    

In [ ]:
def plot_pdf(data, label='', x_axis=''):
    from scipy.stats.kde import gaussian_kde
    from numpy import linspace
    
    # create two subplots
    fig, axs = plt.subplots(1, 2)
    fig.suptitle("Probability Distribution and Historgram "+label)
    
    #create the kernel, which estimates the probability over the values in the array
    kde = gaussian_kde(data)
    
    #these are the values over wich your kernel will be evaluated
    dist_space = linspace(min(data),max(data), 100)

    # plot the results
    axs[0].plot(dist_space, kde(dist_space))
    axs[0].set_ylabel('probability')
    axs[0].set_xlabel(x_axis)
    axs[1].hist(data)
    axs[1].set_ylabel('histogram')
    axs[1].set_xlabel('x_axis')
    plt.show()

In [ ]:
def get_dist(df, var='age', bucket_size=5, to_print=False):
    import math
    
    # create an array that holds the counts of different age groups in bucket_size 
    arr = [];
    for x in range(0,100,bucket_size):
        arr.append(0)
    
    # go through the data, look at each age and update counter for that age group
    for x in df[var]:
        index = math.floor(x/bucket_size)
        arr[index] = arr[index] + 1;

    if to_print:
        age = 0;
        # print the array
        for x in arr:
            print(str(age) + "-" + str(age+bucket_size), ": ", x)
            age = age + bucket_size
    
    return arr

In [ ]:
def find_new_len(len_df_one, dist_one, len_df_two, dist_two):
    import math 
    
    new_len = len_df_one

    for precision in np.arange(1, 0, -0.05):
        new_len = len_df_one
        for i in range(len(dist_one)):
            if dist_one[i] < (new_len*dist_two[i]/len_df_two)*precision:
                if dist_one[i] != 0:
                    new_len = math.floor(dist_one[i]*len_df_two/dist_two[i])
                else:
                    new_len = new_len - (new_len*dist_two[i]/len_df_two)
        if new_len >= 50:
            print("New Length: ", new_len, " with Precision: ", precision)
            break
            
    return new_len

In [ ]:
def match_dist(df_to, df_from, var='age', var_1 = 'subject', bucket_size=3, new_len=None):
    import math 
    
    df_new = pd.DataFrame([])
    
    # sort the dataframes
    df_to = df_to.sort_values(by=var)
    df_from = df_from.sort_values(by=var)
    
    # get the distributions from both
    dist_to = get_dist(df_to, bucket_size=bucket_size)
    dist_from = get_dist(df_from, bucket_size=bucket_size)
    
    index = 0
    i = 0
    rand_subj = []
    
    if new_len:
        df_new_length = new_len
    else:
        # calculate the size of the new dataset that would allow for the same distribution as the dataset being mimicked
        df_new_length = find_new_len(len(df_to), dist_to, len(df_from), dist_from)
    
    # go through the number of subjects in the age range of the resampling data
    for x in np.arange(0, 100, bucket_size):
        subj_bucket = []
        
        # get all the subjects in the age range of the resampling data
        for j in range(dist_to[i]):
            subj_bucket.append((df_to[var_1])[index+j])
        
        index = index + dist_to[i]
        
        # get the number of subjects in the data that is being mimicked
        num_rand = math.floor(df_new_length * (dist_from[i]/len(df_from)))
        #print(len(subj_bucket), num_rand)
        
        if not len(subj_bucket) == 0:
            # choose random subjects from subject bucket
            # the number of random subjects chosen depends on the distributions
            rand_subj = []
            
            if dist_to[i] < num_rand:
                rand_subj = np.random.choice(subj_bucket, size=dist_to[i], replace=False)
            else:
                rand_subj = np.random.choice(subj_bucket, size=num_rand, replace=False)
        
        for sub in rand_subj:
                temp = pd.DataFrame(df_to[df_to[var_1] == sub])
                df_new = pd.concat([df_new, df_to[df_to[var_1] == sub]])

        # increment index for age distribution array
        i = i+1
    
    return df_new

In [ ]:
def sample(df_to, df_from, title_to='', title_from='', var='age', var_1='subject', bucket_size=3, new_len = None):
    from scipy import stats

    plot_pdf(df_from[var], title_from, var)
    plot_pdf(df_to[var], title_to , var)
    
    # sample from original
    df_sample = match_dist(df_to, df_from, bucket_size = bucket_size, new_len = new_len)
    print("Actual Length: ", len(df_sample))
    print(df_sample.head())
    
    # plot new probability density function
    plot_pdf(df_sample[var], title_to + ":" + title_from, var)
    
    # test if the two distributions would be the same
    print("stats: ", stats.ks_2samp(df_sample[var], df_from[var]))
    
    return df_sample

# Hypothesis #1

Research Question:
If the distribution of ages in training and test sets is causing the poor generalizability of both complex and simple models, then both simple and complex models trained on datasets with similar age distributions to our original training dataset will perform worse than chance when tested on samples from new datasets that mimic the age distribution of our original test dataset.

## Setup

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import subprocess
from datetime import date
import re
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob

In [ ]:
# project directory
project_dir = Path('/data/NNDSP')

# NNDSP data directories
nndsp_bids_dir = Path('/data/NNDSP/bids_2017_07_14_generic')
nndsp_fs_dir = Path('/data/NNDSP/derivatives/fs5.3_subj')
nndsp_bar_dir = Path('/data/NNDSP/derivatives/bar_subj')
nndsp_pheno_file = Path('/data/NNDSP/anal/analysis_notebooks/phenotype_files/NNDSP_famid.csv')

# HCP data directories
hcp_bids_dir = Path('/data/HCP/HCP_900/s3/hcp')
hcp_fs_dir = Path('/data/NNDSP/derivatives/fs_hcp_subj')
hcp_bar_dir = Path('/data/NNDSP/derivatives/bar_hcp_subj')
hcp_pheno_file = Path('/data/NNDSP/nino/HCP_ages.csv')

# NKI data directories
nki_bids_dir = Path('/data/NNDSP/anal/NKI')
nki_fs_dir = Path('/data/NNDSP/derivatives/fs_nki_subj')
nki_bar_dir = Path('/data/NNDSP/derivatives/bar_nki_subj')
nki_pheno_file = Path('/data/NNDSP/anal/analysis_notebooks/phenotype_files/participants.tsv')

# CoRR data directories
corr_bids_dir = Path('/data/DSST/CoRR/bids_corr')
corr_fs_dir = Path('/data/DSST/CoRR/fs_corr')
corr_bar_dir = Path('/data/DSST/CoRR/bar_corr')
corr_pheno_file = Path('/data/DSST/CoRR/phenotype_files/corr_ages.csv')

# SALD data directories
sald_bids_dir = Path('/data/DSST/SALD/bids_sald')
sald_fs_dir = Path('/data/DSST/SALD/fs_sald')
sald_bar_dir = Path('/data/DSST/SALD/bar_sald/baracus')
sald_pheno_file = Path('/data/DSST/SALD/phenotype_files/sub_information.xlsx')

## NNDSP Subject Data

In [ ]:
# merge the subjects that we have fs and add subject number
df_nndsp = pd.DataFrame({'subj_paths' : [x.as_posix() for x in nndsp_fs_dir.glob('sub-*')]})
df_nndsp = df_nndsp.assign(MASKID = [int(Path(x).name[4:]) for x in df_nndsp.subj_paths])
df_nndsp.head()

In [ ]:
# merge the subjects we have fs of with their ages
df_nndsp = pd.merge(df_nndsp, pd.read_csv(nndsp_pheno_file.as_posix()), on='MASKID')

In [ ]:
# rename columns so that we have standardization across datasets
df_nndsp = df_nndsp.rename(index=str, columns = {'MASKID' : 'subject', 'age_at_scan' : 'age'})
df_nndsp = df_nndsp.drop_duplicates(subset='subject', keep='first')
df_nndsp.head()

## HCP Subject Data

In [ ]:
# merge the subjects that we have fs and add subject number
df_hcp = pd.DataFrame({'subj_paths' : [x.as_posix() for x in hcp_fs_dir.glob('sub-*')]})
df_hcp = df_hcp.assign(Subject = [int(Path(x).name[4:]) for x in df_hcp.subj_paths])
df_hcp.head()

In [ ]:
# merge the subjects we have fs of with their ages
df_hcp = pd.merge(df_hcp, pd.read_csv(hcp_pheno_file.as_posix()), on='Subject')

In [ ]:
# rename columns so that we have standardization across datasets
df_hcp = df_hcp.rename(index=str, columns={'Subject' : 'subject', 'Age_in_Yrs' : 'age'})
df_hcp = df_hcp.drop_duplicates(subset='subject', keep='first')
df_hcp.head()

## NKI Subject Data

In [ ]:
# merge the subjects that we have fs and add subject number
df_nki = pd.DataFrame({'subj_paths' : [x.as_posix() for x in nki_fs_dir.glob('sub-*')]})
df_nki = df_nki.assign(participant_id = [Path(x).name[4:13] for x in df_nki.subj_paths])
df_nki.head()

In [ ]:
# merge the subjects that we have fs with age
df_nki = pd.merge(df_nki, pd.read_csv(nki_pheno_file.as_posix(), sep='\t'), on='participant_id')

In [ ]:
# rename so that we have standardization across datasets
df_nki = df_nki.rename(index=str, columns={'participant_id' : 'subject', 'age' : 'age'})
df_nki = df_nki.drop_duplicates(subset='subject', keep='first')
df_nki.head()

## CoRR Subject Data

In [ ]:
# merge the subjects that we have fs and add subject number
df_corr = pd.DataFrame({'subj_paths' : [x.as_posix() for x in corr_fs_dir.glob('sub-*')]})
df_corr = df_corr.assign(SUBID = [int(Path(x).name[4:11]) for x in df_corr.subj_paths])
df_corr.head()

In [ ]:
# merge subjects with ages 
df_corr = pd.merge(df_corr, pd.read_csv(corr_pheno_file.as_posix()), on='SUBID')

In [ ]:
# rename so that we have standardization across datasets
df_corr = df_corr.rename(index=str, columns={'SUBID' : 'subject', 'AGE_AT_SCAN_1' : 'age'})
df_corr = df_corr.sort_values(by='subj_paths')
df_corr = df_corr.drop_duplicates(subset='subject', keep='first')
df_corr.age = pd.to_numeric(df_corr.age, errors='coerce').fillna(0).astype(np.int64)
df_corr.head()

## SALD Subject Data

In [ ]:
# complete once we have SALD FS files
df_sald = pd.DataFrame({'subj_paths': [x.as_posix() for x in sald_fs_dir.glob('sub-*')]})
df_sald = df_sald.assign(Sub_ID = [int(Path(x).name[4:]) for x in df_sald.subj_paths])
df_sald.head()

In [ ]:
# merge phenotype file with dataframe
df_sald = pd.merge(df_sald, pd.read_excel(sald_pheno_file.as_posix()), on='Sub_ID')

In [ ]:
df_sald = df_sald.rename(index = str, columns = {'Sub_ID': 'subject', 'Age': 'age'})
df_sald = df_sald.sort_values(by='subject')
df_sald = df_sald.drop_duplicates(subset='subject', keep='first')
df_sald.age = pd.to_numeric(df_sald.age, errors='coerce').fillna(0).astype(np.int64)
df_sald.head()

## Sample NKI:NNDSP

In [ ]:
df_nki_nndsp = sample(df_nki, df_nndsp, title_to='NKI', title_from='NNDSP')

## Sample CoRR:NNDSP

In [ ]:
df_corr_nndsp = sample(df_corr, df_nndsp, title_to='CoRR', title_from='NNDSP', bucket_size=3)

## Sample SALD:NNDSP

In [ ]:
df_sald_nndsp = sample(df_sald, df_nndsp, title_to='SALD', title_from='NNDSP', bucket_size=10, new_len=65)

## Sample NKI:HCP

In [ ]:
df_nki_hcp = sample(df_nki, df_hcp, title_to='NKI', title_from='HCP', bucket_size=2)

## Sample CoRR:HCP

In [ ]:
df_corr_hcp = sample(df_corr, df_hcp, title_to='CoRR', title_from='HCP', bucket_size=3)

## Sample SALD:HCP

In [ ]:
df_sald_hcp = sample(df_sald, df_hcp, title_to='SALD', title_from='HCP', bucket_size=2, new_len=65)

# Train Test Hypothesis

In [ ]:
%run /data/NNDSP/anal/analysis_notebooks/follow_up_analysis/util_models.ipynb

In [ ]:
hcp_mae = {}
nki2hcp_mae = {}
corr2hcp_mae = {}
sald2hcp_mae = {}

## Train NNDSP

In [ ]:
scores_nndsp, nndsp_complex_train, nndsp_complex_test, nndsp_complex_pipes = complex_model(df_nndsp, nndsp_bar_dir)

In [ ]:
nndsp_simple_train, nndsp_simple_test, nndsp_simple_pipe = simple_model(df_nndsp, nndsp_bar_dir, 
                                                                        model=True, 
                                                                        model_train = nndsp_complex_train, 
                                                                        model_test=nndsp_complex_test)

### Test on HCP

In [ ]:
hcp_mae['Complex on NNDSP'] = complex_test(df_hcp, hcp_bar_dir, nndsp_complex_pipes, data='HCP')

In [ ]:
hcp_mae['Simple on NNDSP'] = simple_test(df_hcp, hcp_bar_dir, nndsp_simple_pipe, data='HCP')

### Test on NKI:HCP

In [ ]:
nk2hcp_mae['Complex on NNDSP'] = complex_test(df_nki_hcp, nki_bar_dir, nndsp_complex_pipes, data='NKI:HCP', is_int=False)

In [ ]:
nki2hcp_mae['Simple on NNDSP'] = simple_test(df_nki_hcp, nki_bar_dir, nndsp_simple_pipe, data='NKI:HCP', is_int=False)

### Test on CORRS:HCP

In [ ]:
#corr2hcp_mae['Complex on NNDSP'] complex_test(df_nki_hcp, nki_bar_dir, nndsp_complex_pipes, data='NKI:HCP', is_int=False)

In [ ]:
#corr2hcp_mae['Simple on NNDSP] simple_test(df_nki_hcp, nki_bar_dir, nndsp_simple_pipe, data='NKI:HCP', is_int=False)

### Test on SALD:HCP

In [ ]:
sald2hcp_mar['Complex on NNDSP'] = complex_test(df_sald_hcp.drop_duplicates(), sald_bar_dir, nndsp_complex_pipes, data='SALD:HCP')

In [ ]:
sal2hcp_mae['Simple on NNDSP'] = simple_test(df_sald_hcp, sald_bar_dir, nndsp_simple_pipe, data='SALD:HCP')

## Train NKI:NNDSP

In [ ]:
df_nki_nndsp.head()

In [ ]:
scores_nki2nndsp, nki2nndsp_complex_train, nki2nndsp_complex_test, nki2nndsp_complex_pipes = complex_model(df_nki_nndsp, 
                                                                                                           nki_bar_dir, 
                                                                                                           is_int = False)

In [ ]:
nki2nndsp_simple_train, nki2nndsp_simple_test, nki2nndsp_simple_pipe = simple_model(df_nki_nndsp, nki_bar_dir, 
                                                                        model=True, 
                                                                        model_train = nki2nndsp_complex_train, 
                                                                        model_test=nki2nndsp_complex_test,
                                                                                   is_int=False)

### Test on HCP

In [ ]:
hcp_mae['Complex on NKI:NNDSP'] = complex_test(df_hcp, hcp_bar_dir, nki2nndsp_complex_pipes, data='HCP', is_int=True)

In [ ]:
hcp_mae['Simple on NKI:NNDSP'] = simple_test(df_hcp, hcp_bar_dir, nki2nndsp_simple_pipe, data='HCP')

### Test on CoRR:HCP

In [ ]:
#corr2hcp_mae['Complex on NKI:NNDSP'] = complex_test(df_corr_hcp, corr_bar_dir, nki2nndsp_complex_pipes, data='CoRR:HCP')

In [ ]:
#corr2hcp_mae['Simple on NKI:NNDSP'] = simple_test(df_corr_hcp, corr_bar_dir, nki2nndsp_simple_pipe, data='CoRR:HCP')

### Test on SALD:HCP

In [ ]:
sald2hcp_mae['Complex on NKI:NNDSP'] = complex_test(df_sald_hcp, sald_bar_dir, nki2nndsp_complex_pipes, data='SALD:HCP')

In [ ]:
sald2hcp_mae['Simple on NKI:NNDSP'] = simple_test(df_sald_hcp, sald_bar_dir, nki2nndsp_simple_pipe, data='SALD:HCP')

## Train on CoRR:NNDSP

In [ ]:
# scores_corr2nndsp, corr2nndsp_complex_train, corr2nndsp_complex_test, corr2nndsp_complex_pipes = complex_model(df_corr_nndsp, 
#                                                                                                            corr_bar_dir, 
#                                                                                                            is_int = True)

In [ ]:
# corr2nndsp_simple_train, corr2nndsp_simple_test, corr2nndsp_simple_pipe = simple_model(df_corr_nndsp, corr_bar_dir, 
#                                                                         model=True, 
#                                                                         model_train = corr2nndsp_complex_train, 
#                                                                         model_test=corr2nndsp_complex_test,
#                                                                                    is_int=True)

### Test on HCP

In [ ]:
# hcp_mae['Complex on CoRR:NNDSP'] = complex_test(df_hcp, hcp_bar_dir, corr2nndsp_complex_pipes, data='HCP', is_int=True)

In [ ]:
# hcp_mae['Simple on CoRR:NNDSP'] = simple_test(df_hcp, hcp_bar_dir, corr2nndsp_simple_pipe, data='HCP')

### Test on NKI:HCP

In [ ]:
# nki2hcp_mae['Complex on CoRR:NNDSP'] = complex_test(df_nki_hcp, nki_bar_dir, corr2nndsp_complex_pipes, data='NKI:HCP', is_int=False)

In [ ]:
# nki2hcp_mae['Simple on CoRR:NNDSP'] = simple_test(df_nki_hcp, nki_bar_dir, corr2nndsp_simple_pipe, data='NKI:HCP', is_int = False)

### Test on SALD:HCP

In [ ]:
# sald2hcp_mae['Complex on CoRR:NNDSP'] = complex_test(df_sald_hcp, sald_bar_dir, corr2nndsp_complex_pipes, data='SALD:HCP', is_int=True)

In [ ]:
# sald2hcp_mae['Simple on CoRR:NNDSP'] = simple_test(df_sald_hcp, sald_bar_dir, corr2nndsp_simple_pipe, data='SALD:HCP', is_int = True)

# Statistical Tests

In [ ]:
%run /data/NNDSP/anal/analysis_notebooks/follow_up_analysis/util_stats.ipynb

## Test on HCP

In [ ]:
hcp_rand_ages = random_ages(df_hcp)
plot_rand_hist(hcp_rand_ages, hcp_mae, title='HCP',
              set_context='notebook', fig_tuple=(10, 7), legend=True)
get_percentile(hcp_rand_ages, hcp_mae)

## Test NKI:HCP

In [ ]:
nki2hcp_rand_ages = random_ages(df_nki_hcp)
plot_rand_hist(nki2hcp_rand_ages, nki2hcp_mae, title='NKI:HCP',
              set_context='notebook', fig_tuple=(10, 7), legend=True)
get_percentile(nki2hcp_rand_ages, nki2hcp_mae)

## Test CoRR:HCP

In [ ]:
corr2hcp_rand_ages = random_ages(df_corr_hcp)
plot_rand_hist(corr2hcp_rand_ages, corr2hcp_mae, title='CoRR:HCP',
              set_context='notebook', fig_tuple=(10, 7), legend=True)
get_percentile(corr2hcp_rand_ages, corr2hcp_mae)

## SALD:HCP

In [ ]:
sald2hcp_rand_ages = random_ages(df_sald_hcp)
plot_rand_hist(sald2hcp_rand_ages, sald2hcp_mae, title='SALD:HCP',
              set_context='notebook', fig_tuple=(10, 7), legend=True)
get_percentile(sald2hcp_rand_ages, sald2hcp_mae)